In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import matplotlib.cm as cm
from scipy.misc import imresize
import cv2
import pandas as pd
import scipy.io as io

In [2]:
def data_extraction(path):
    data_l, data_r, data_all = [], [], []
    nor_data = io.loadmat(path)
    temp = nor_data['data']
    r = temp['right']
    l = temp['left']
    r_temp = r[0,0]
    l_temp = l[0,0]
    data_r.append(r_temp['image'][0][0])
    data_r.append(r_temp['gaze'][0][0])
    data_r.append(r_temp['pose'][0][0])
    data_l.append(l_temp['image'][0][0])
    data_l.append(l_temp['gaze'][0][0])
    data_l.append(l_temp['pose'][0][0])
    for i in range(3):
        data_all.append(data_r[i])   #0 -> r_img, 1-> l_img, 2-> r_gaze, 3->l_gaze..
        data_all.append(data_l[i])
    return data_all

In [3]:
def label_extraction(path, screen_path):
    txt = pd.read_csv(path, sep = ' ', header= None)
    screen = io.loadmat(screen_path)
    df_anno = pd.DataFrame(txt)
    data_frame = pd.DataFrame()
    data_frame['Screen_x'] = df_anno[:][24] / screen['width_pixel'][0][0]
    data_frame['Screen_y'] = df_anno[:][25] / screen['height_pixel'][0][0]
    return data_frame

In [4]:
def path_optimization(day_list, path):
    all_path = []
    cnt = 0
    for i in range(len(day_list)): # 15번 
        temp = []
        for j in range(day_list[i]):
            temp.append(path[cnt])
            cnt += 1
        all_path.append(temp)
    return all_path

day_list = [39, 69, 39, 65, 25, 38, 62, 56, 47, 20, 16, 19, 7, 12, 7]  
data_path = glob('/home/spow12/Codes/Data/MPIIGaze/Data//Normalized/*/*')# 521개
data_path.sort()    #Sort for prevent index error
data_path = path_optimization(day_list, data_path)
#data_path = path_optimization(glob('C:/MPIIGaze/MPIIGaze/Data/Normalized/*/*'))
label_path = glob('/home/spow12/Codes/Data/MPIIGaze/Data/Original/*/*/annotation.txt') # 521개
label_path.sort()
label_path = path_optimization(day_list, label_path)
#label_path = path_optimzation(glob('C:/MPIIGaze/MPIIGaze/Data/Original/*/*/annotation.txt'))
screen_size_path = glob('/home/spow12/Codes/Data/MPIIGaze/Data/Original/*/Calibration/screenSize.mat') # 15개

for i in range(15): # 총 15명
    data = [] # 사람별로 나눈 데이터
    label = [] # 사람별로 나눈 라벨   
    for j in range(day_list[i]): # 각 사람에 맞는 day만큼
        data.append(data_extraction(data_path[i][j]))
        label.append(label_extraction(label_path[i][j], screen_size_path[i]))
    exec('p%d_data = data' % (i))
    exec('p%d_label = label' % (i))

In [5]:
def make_dataset_img_label(image_shape, label_shape, data_list, label_list):
    length = len(data_list[0])
    left_img = np.zeros([length,image_shape])
    right_img = np.zeros([length,image_shape])
    label = np.zeros([length,label_shape])
    for n in range(length):
        left_img[n, :] = data_list[1][n].reshape(image_shape)      
        right_img[n, :] = data_list[0][n].reshape(image_shape)
        label[n, :] = [label_list['Screen_x'][n], label_list['Screen_y'][n]]
        
    return right_img, left_img, label

def make_dataset_gaze_pose(data_shape, data_list):
    length = len(data_list[0])
    left_pose = np.zeros([length,data_shape])
    right_pose = np.zeros([length,data_shape])
    left_gaze = np.zeros([length,data_shape])
    right_gaze = np.zeros([length,data_shape])
    for n in range(length):
        left_gaze[n, :] = data_list[3][n]
        right_gaze[n, :] = data_list[2][n]
        left_pose[n, :] = data_list[5][n]
        right_pose[n, :] = data_list[4][n]
    return left_gaze, right_gaze, left_pose, right_pose

In [6]:
Batch_RE_img_data = []
Batch_LE_img_data = []
#Batch_LE_gaze_data = []
#Batch_RE_gaze_data = [] 
#Batch_LE_pose_data = []
#Batch_RE_pose_data = []
Batch_label = []
for i in range(15):
    for j in range(day_list[i]): #days list in Data  po -> 39days  ..
        exec('RE_img_data, LE_img_data, dataset_label = make_dataset_img_label(36 * 60, 2, p{0}_data[{1}], p{2}_label[{3}])'
             .format(i, j, i, j))
        #exec('LE_gaze_data, RE_gaze_data, LE_pose_data, RE_pose_data = make_dataset_gaze_pose(3, p{0}_data[{1}])'.format(i, j))
        Batch_RE_img_data.append(RE_img_data)
        Batch_LE_img_data.append(LE_img_data)
#         Batch_LE_gaze_data.append(LE_gaze_data)
#         Batch_RE_gaze_data.append(RE_gaze_data)
#         Batch_LE_pose_data.append(LE_pose_data)
#         Batch_RE_pose_data.append(RE_pose_data)
        Batch_label.append(dataset_label)

# Make graph for eye images

In [7]:
def model_for_image(X_img, keep_prob, name):
    with tf.name_scope("Image_layer_1" + name) as scope:
        W1 = tf.Variable(tf.random_normal([4, 4, 1, 5], stddev=0.01))
        L1 = tf.nn.conv2d(X_img, W1, strides=[1, 2, 2, 1], padding='SAME')
        L1 = tf.nn.relu(L1)

    with tf.name_scope("Image_layer_2" + name) as scope:
        W2 = tf.Variable(tf.random_normal([2, 2, 5, 10], stddev=0.01))
        L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding='SAME')
        L2 = tf.nn.relu(L2)
        L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
        L2_flat = tf.reshape(L2, [-1, 5 * 8 * 10])
    

    with tf.name_scope("Image_layer_3" + name) as scope:
        W3 = tf.get_variable("W3_"+name ,shape= [400, 256],
                             initializer= tf.contrib.layers.xavier_initializer())
        b3 = tf.Variable(tf.random_normal([256]))
        L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
        L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

    with tf.name_scope("Image_layer_4" + name) as scope:
        W4 = tf.get_variable("W4_"+name ,shape= [256, 128],
                             initializer= tf.contrib.layers.xavier_initializer())
        b4 = tf.Variable(tf.random_normal([128]))
        L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
        L4 = tf.nn.dropout(L4, keep_prob= keep_prob)
    
    with tf.name_scope("Image_layer_5" + name) as scope:
        W5 = tf.get_variable("W5_"+name ,shape= [128, 64],
                             initializer= tf.contrib.layers.xavier_initializer())
        b5 = tf.Variable(tf.random_normal([64]))
        logits = tf.matmul(L4, W5) + b5
    
    return logits

# Make graph for other datas

In [8]:
def model_for_others(X, name):
    with tf.name_scope("Layer_1" + name) as scope:
        W1 = tf.get_variable("W1_"+name ,shape= [3, 3],
                             initializer= tf.contrib.layers.xavier_initializer())
        b1 = tf.Variable(tf.random_normal([3]))
        L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    with tf.name_scope("Layer_2" + name) as scope:
        W2 = tf.get_variable("W2_"+name ,shape= [3, 3],
                             initializer= tf.contrib.layers.xavier_initializer())
        b2 = tf.Variable(tf.random_normal([3]))
        logits = tf.matmul(L1, W2) + b2
    
    return logits

# Define variable and make graphs using function

In [9]:
learning_rate = 0.001

Y = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)

X_left = tf.placeholder(tf.float32, [None, 2160])
X_right = tf.placeholder(tf.float32, [None, 2160])
X_img_left = tf.reshape(X_left, [-1, 36, 60, 1])   
X_img_right = tf.reshape(X_right, [-1, 36, 60, 1]) 

# X_gaze_r = tf.placeholder(tf.float32, [None, 3])
# X_gaze_l = tf.placeholder(tf.float32, [None, 3])
# X_pose_r = tf.placeholder(tf.float32, [None, 3])
# X_pose_l = tf.placeholder(tf.float32, [None, 3])

LE_img = model_for_image(X_img_left, keep_prob, 'left_eye')  #left eye imgae
RE_img = model_for_image(X_img_right, keep_prob, 'right_eye')#right eye image
# LE_pose = model_for_others(X_pose_l, 'left_pose')
# LE_gaze = model_for_others(X_gaze_l, 'left_gaze')
# RE_pose = model_for_others(X_pose_r, 'right_pose')
# RE_gaze = model_for_others(X_gaze_r, 'right_gaze')

W0912 00:27:09.851532 140313676670720 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0912 00:27:09.861533 140313676670720 deprecation.py:506] From <ipython-input-7-dc5994c33a26>:21: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Integrate the gaze graph

In [10]:
# with tf.name_scope("Layer_gaze_merged") as scope:
#     LE_gaze_W = tf.get_variable("LE_gaze_W" ,shape= [3, 3],
#                              initializer= tf.contrib.layers.xavier_initializer())

#     RE_gaze_W = tf.get_variable("RE_gaze_W" ,shape= [3, 3],
#                              initializer= tf.contrib.layers.xavier_initializer())

#     layer_gaze = tf.nn.relu(tf.matmul(LE_gaze, LE_gaze_W) + tf.matmul(RE_gaze, RE_gaze_W))

# Integrate the pose layer

In [11]:
# with tf.name_scope("Layer_pose_merged") as scope:
#     LE_pose_W = tf.get_variable("LE_pose_W" ,shape= [3, 3],
#                              initializer= tf.contrib.layers.xavier_initializer())

#     RE_pose_W = tf.get_variable("RE_pose_W" ,shape= [3, 3],
#                              initializer= tf.contrib.layers.xavier_initializer())

#     layer_pose = tf.nn.relu(tf.matmul(LE_pose, LE_pose_W) + tf.matmul(RE_pose, RE_pose_W))

# Integrate the Image layer

In [12]:
with tf.name_scope("Layer_Image_merged_1") as scope:
    LE_W_img = tf.get_variable("LE_W_img" ,shape= [64, 32],
                             initializer= tf.contrib.layers.xavier_initializer())
    RE_W_img = tf.get_variable("RE_W_img" ,shape= [64, 32],
                             initializer= tf.contrib.layers.xavier_initializer())
    layer_img = tf.nn.relu(tf.matmul(LE_img, LE_W_img) + tf.matmul(RE_img, RE_W_img))

with tf.name_scope("Layer_Image_merged_2") as scope:
    img_W = tf.get_variable("img_W" ,shape= [32, 16],
                             initializer= tf.contrib.layers.xavier_initializer())
    img_b = tf.Variable(tf.random_normal([16]))
    layer2_img = tf.nn.relu(tf.matmul(layer_img, img_W) + img_b)

with tf.name_scope("Layer_Image_merged_3") as scope:
    img_W2 = tf.get_variable("img_W2" ,shape= [16, 8],
                             initializer= tf.contrib.layers.xavier_initializer())
    img_b2 = tf.Variable(tf.random_normal([8]))
    layer3_img= tf.nn.relu(tf.matmul(layer2_img, img_W2) + img_b2)

with tf.name_scope("Layer_Image_merged_Last") as scope:
#     Final_W_img = tf.get_variable("Final_W_img" ,shape= [8, 3],
#                              initializer= tf.contrib.layers.xavier_initializer())
#     Final_b_img = tf.Variable(tf.random_normal([3]))
#     Final_layer_img= tf.nn.relu(tf.matmul(layer3_img, Final_W_img) + Final_b_img)
    Final_W_img = tf.get_variable("Final_W_img" ,shape= [8, 2],
                             initializer= tf.contrib.layers.xavier_initializer())
    Final_b_img = tf.Variable(tf.random_normal([2]))
    logits =  tf.matmul(layer3_img, Final_W_img)

In [13]:
# with tf.name_scope("Layer") as scope:
#     # Weight and bias for img layer
#     All_W_img = tf.get_variable("All_W_img" ,shape= [3, 2],
#                              initializer= tf.contrib.layers.xavier_initializer())
#     # Weight and bias for pose layer
#     All_W_pose = tf.get_variable("All_W_pose" ,shape= [3, 2],
#                              initializer= tf.contrib.layers.xavier_initializer())
#     # Weight and bias for gaze layer
#     All_W_gaze = tf.get_variable("All_W_gaze" ,shape= [3, 2],
#                              initializer= tf.contrib.layers.xavier_initializer())
#     logits = (tf.matmul(Final_layer_img, All_W_img) 
#               + tf.matmul(layer_pose, All_W_pose)
#               + tf.matmul(layer_gaze, All_W_gaze))
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.square(logits-Y))

with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

for var in tf.trainable_variables():
    tf.summary.histogram(var.op.name, var)

summary = tf.summary.merge_all()
saver = tf.train.Saver()

In [14]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('../temp/graph/graph_summary')
    writer.add_graph(sess.graph)

    print('Learning started.')
    for i in range(521):  #Batch number
        Batch_size = len(Batch_RE_img_data)
        feed_dict = {X_right: Batch_RE_img_data[i],
                         X_left: Batch_LE_img_data[i],
                         Y:Batch_label[i], keep_prob: 0.7}
        s, _ = sess.run([summary, optimizer], feed_dict=feed_dict)
        c = sess.run(cost, feed_dict=feed_dict)
        writer.add_summary(s, global_step=i)
        print('step: {0}, cost: {1}'.format(i, c))
        save_path = saver.save(sess,'../temp/variables/model.ckpt') 
        
print('Learning Finished!')

Learning started.
step: 0, cost: 0.7690896987915039
step: 1, cost: 0.46866631507873535
step: 2, cost: 0.2511560916900635
step: 3, cost: 0.4830506443977356
step: 4, cost: 0.29519417881965637
step: 5, cost: 0.25252676010131836
step: 6, cost: 0.24574269354343414
step: 7, cost: 0.2780194580554962
step: 8, cost: 0.2571008801460266
step: 9, cost: 0.22935278713703156
step: 10, cost: 0.2190808653831482
step: 11, cost: 0.20605072379112244
step: 12, cost: 0.19693447649478912
step: 13, cost: 0.22212575376033783
step: 14, cost: 0.20544682443141937
step: 15, cost: 0.21702444553375244
step: 16, cost: 0.2017958164215088
step: 17, cost: 0.194117471575737
step: 18, cost: 0.17154739797115326
step: 19, cost: 0.20084431767463684
step: 20, cost: 0.1968308836221695
step: 21, cost: 0.1990969181060791
step: 22, cost: 0.18175657093524933
step: 23, cost: 0.1864393651485443
step: 24, cost: 0.17940406501293182
step: 25, cost: 0.1758061647415161
step: 26, cost: 0.17980556190013885
step: 27, cost: 0.176109507679939

step: 227, cost: 0.06935308128595352
step: 228, cost: 0.07609367370605469
step: 229, cost: 0.07250401377677917
step: 230, cost: 0.06999155879020691
step: 231, cost: 0.06724565476179123
step: 232, cost: 0.08293311297893524
step: 233, cost: 0.0648312121629715
step: 234, cost: 0.06680280715227127
step: 235, cost: 0.06466595083475113
step: 236, cost: 0.060910556465387344
step: 237, cost: 0.0938713327050209
step: 238, cost: 0.09621425718069077
step: 239, cost: 0.09265151619911194
step: 240, cost: 0.1077377125620842
step: 241, cost: 0.09509018063545227
step: 242, cost: 0.07693219184875488
step: 243, cost: 0.08783073723316193
step: 244, cost: 0.08222126215696335
step: 245, cost: 0.08672964572906494
step: 246, cost: 0.08115910738706589
step: 247, cost: 0.08034137636423111
step: 248, cost: 0.08227182924747467
step: 249, cost: 0.08491094410419464
step: 250, cost: 0.08505649864673615
step: 251, cost: 0.08007843792438507
step: 252, cost: 0.07729148864746094
step: 253, cost: 0.07541247457265854
ste

step: 453, cost: 0.09119074791669846
step: 454, cost: 0.09651841223239899
step: 455, cost: 0.06832385808229446
step: 456, cost: 0.08733825385570526
step: 457, cost: 0.11669967323541641
step: 458, cost: 0.11256285011768341
step: 459, cost: 0.09348117560148239
step: 460, cost: 0.04552656412124634
step: 461, cost: 0.0858067125082016
step: 462, cost: 0.07444082200527191
step: 463, cost: 0.06945180147886276
step: 464, cost: 0.08619771152734756
step: 465, cost: 0.062236953526735306
step: 466, cost: 0.07181594520807266
step: 467, cost: 0.08063165843486786
step: 468, cost: 0.06671218574047089
step: 469, cost: 0.07547857612371445
step: 470, cost: 0.08639656752347946
step: 471, cost: 0.054486293345689774
step: 472, cost: 0.067299023270607
step: 473, cost: 0.10767562687397003
step: 474, cost: 0.05677814036607742
step: 475, cost: 0.07506375759840012
step: 476, cost: 0.04965883865952492
step: 477, cost: 0.05007896572351456
step: 478, cost: 0.05518937483429909
step: 479, cost: 0.05407803878188133
st

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    predictions = logits.eval(feed_dict = {
            X_right: Batch_RE_img_data[39],
            X_left: Batch_LE_img_data[39],
            keep_prob: 1})

In [16]:
Batch_label[39]

array([[0.8046875 , 0.86      ],
       [0.9375    , 0.585     ],
       [0.32578125, 0.62625   ],
       ...,
       [1.015625  , 0.2725    ],
       [0.90703125, 0.625     ],
       [0.6109375 , 0.2925    ]])

In [17]:
predictions

array([[-4.662403 ,  1.9034401],
       [-4.6782975,  1.9079642],
       [-4.66164  ,  1.9132535],
       ...,
       [-4.60349  ,  1.9124138],
       [-4.61962  ,  1.9010751],
       [-4.592382 ,  1.9093984]], dtype=float32)